In [1]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [3]:
class Network:
    def __init__(self, layers):
        # layers/activations
        self.input = np.random.rand(4, 1)
        self.h1 = np.random.rand(4, 1)
        self.h2 = np.random.rand(3, 1)
        self.output = np.random.rand(3, 1)
        
        # weights 
        self.w_1 = np.random.rand(4, 4)
        self.w_2 = np.random.rand(3, 4)
        self.w_3 = np.random.rand(3, 3)
        
        # biases 
        self.b_1 = np.random.rand(4, 1)
        self.b_2 = np.random.rand(3, 1)
        self.b_3 = np.random.rand(3, 1)
        
    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))

    def softmax(self, activations):
        return np.exp(activations) / np.sum(np.exp(activations))
    
    def feed_forward(self, X):
        # input
        self.input = np.reshape(X, (-1,1))
        
        # input to h1
        h1_activations = self.sigmoid(np.dot(self.w_1, self.input) + self.b_1)
        self.h1 = h1_activations


        # h1 to h2
        h2_activations = self.sigmoid(np.dot(self.w_2, h1_activations) + self.b_2)
        self.h2 = h2_activations

        # h2 to output 
        output_activations = (np.dot(self.w_3, h2_activations) + self.b_3)
        self.output = self.sigmoid(output_activations)

        return (self.output, max(self.output))
    
    def back_prop(self, y, learning_rate=0.1):
        y = np.reshape(y, (-1,1))
        
        output_error = (2 * (self.output-y)) * ((self.sigmoid(self.output)*(1-self.sigmoid(self.output))))
        w_3 = -learning_rate * (output_error.dot(self.h2.T))
        b_3 = -learning_rate *output_error
        
        
        h2_error = w_3.T.dot(output_error) * ((self.sigmoid(self.h2)*(1-self.sigmoid(self.h2))))
        w_2 = -learning_rate * (h2_error.dot(self.h1.T))
        b_2 = -learning_rate * h2_error
        
        h1_error = w_2.T.dot(h2_error) * ((self.sigmoid(self.h1)*(1-self.sigmoid(self.h1))))
        w_1 = -learning_rate * (np.dot(h1_error, self.input.T))
        b_1 = -learning_rate * h1_error
        
        self.w_1 += w_1
        self.w_2 += w_2
        self.w_3 += w_3
        
        self.b_1 += b_1
        self.b_2 += b_2
        self.b_3 += b_3
        return 0
       

    def train(self, X, Y, itterations):        
        y_dummies = pd.get_dummies(Y)
        data = pd.concat([X, y_dummies], axis=1)
        for i in range(itterations):
            for index, row in data.iterrows():
                # select the label
                y = row.tolist()[4:]
                # select the x
                x = row.tolist()[:4] 
                output_activations, prediction = self.feed_forward(x)
                self.back_prop(y)    

In [4]:
list(X_train.iloc[0])

[5.5, 2.4, 3.7, 1.0]

In [5]:
# number of neurons at each layers input, h1, h2, output
layers = (len(X_train.iloc[0]), 4, 3, 3)

In [6]:
new_class = Network(layers=layers)

In [7]:
new_class.train(X_train, y_train, 20)